In [48]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import pickle

In [2]:
from matplotlib import rcParams
from matplotlib import cm, colors, _cm
rcParams['font.family'] = 'NanumGothic'

In [4]:
import urllib
import json
from time import time

In [89]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [5]:
import sqlite3

In [6]:
kb_db = sqlite3.connect("./KB.db")

In [8]:
def getAddrInfo( code, type='SGG'  ):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?bjd_code=%d&bjd_gbn=%s" %(code,type) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])

In [9]:
def getAptInfo(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptList.do?bjd_code=%d" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return pd.DataFrame(data[u'resultList'])

In [10]:
gu_info = getAddrInfo(11)

In [12]:
values=gu_info.apply(lambda x : u"(%d, %s, '%s')" % (11, x['CODE'], x['CODE_VALUE'] ), axis=1)
values

0      (11, 680, '강남구')
1      (11, 740, '강동구')
2      (11, 305, '강북구')
3      (11, 500, '강서구')
4      (11, 620, '관악구')
5      (11, 215, '광진구')
6      (11, 530, '구로구')
7      (11, 545, '금천구')
8      (11, 350, '노원구')
9      (11, 320, '도봉구')
10    (11, 230, '동대문구')
11     (11, 590, '동작구')
12     (11, 440, '마포구')
13    (11, 410, '서대문구')
14     (11, 650, '서초구')
15     (11, 200, '성동구')
16     (11, 290, '성북구')
17     (11, 710, '송파구')
18     (11, 470, '양천구')
19    (11, 560, '영등포구')
20     (11, 170, '용산구')
21     (11, 380, '은평구')
22     (11, 110, '종로구')
23      (11, 140, '중구')
24     (11, 260, '중랑구')
dtype: object

In [13]:
insert_sql = u"insert into addr_gu_code_tbl( si_code, gu_code, name) values %s ;" % (u",".join(values))

In [15]:
kb_db.execute(insert_sql)

In [16]:
kb_db.commit()

In [17]:
city_info=pd.read_sql("select * from addr_gu_code_tbl;",kb_db)

In [18]:
city_info.head()

,si_code,gu_code,name
0,11,680,강남구
1,11,740,강동구
2,11,305,강북구
3,11,500,강서구
4,11,620,관악구


In [19]:
for si,gu in city_info[['si_code','gu_code']].itertuples(index=False):
    code = "%d%d" % (si,gu)
    info = getAddrInfo(int(code),'EMD')
    values=info.apply(lambda x : u"(%d, %d, %s, '%s')" % (si,gu, x['CODE'], x['CODE_VALUE'] ), axis=1)
    insert_sql = u"insert into addr_dong_code_tbl( si_code, gu_code, dong_code,name) values %s ;" % (u",".join(values))
    kb_db.execute(insert_sql)
    kb_db.commit()
    print ("%s code inserted(%d)" % (code, info.count()[0]))

11680 code inserted(14)
11740 code inserted(9)
11305 code inserted(4)
11500 code inserted(13)
11620 code inserted(3)
11215 code inserted(7)
11530 code inserted(10)
11545 code inserted(3)
11350 code inserted(5)
11320 code inserted(4)
11230 code inserted(10)
11590 code inserted(9)
11440 code inserted(26)
11410 code inserted(20)
11650 code inserted(10)
11200 code inserted(17)
11290 code inserted(39)
11710 code inserted(13)
11470 code inserted(3)
11560 code inserted(34)
11170 code inserted(36)
11380 code inserted(11)
11110 code inserted(87)
11140 code inserted(74)
11260 code inserted(6)


In [21]:
town_info=pd.read_sql("select * from addr_dong_code_tbl;",kb_db)
town_info.head()

,si_code,gu_code,dong_code,name
0,11,680,103,개포동
1,11,680,108,논현동
2,11,680,106,대치동
3,11,680,118,도곡동
4,11,680,105,삼성동


In [22]:
def getAptDetailInfo(code):
    response = urllib.request.urlopen("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=%s" % (code) )
    data = json.loads(response.read().decode('utf-8'))
    return data

In [26]:
def extractAptInfo(a):
    info = []
    for name in ['KAPT_CODE', 'KAPT_DONG_CNT','KAPT_USEDATE', 'KAPTD_WTIMESUB', 'SUBWAY_STATION', 'SUBWAY_LINE', 'CODE_HALL', 'KAPT_BCOMPANY' ] :
        info.append(a[u'resultMap_kapt'][name]);
    cnt = 0
    for x in a[u'resultMap_kapt_areacnt']:
        if( type(x[u'KAPTDA_CNT']) is int ):
            info.append(x[u'KAPTDA_CNT'])
        else :
            info.append(0)
#           cnt = cnt + x[u'KAPTDA_CNT']
 #   info.append(cnt)
    return info

In [27]:
for code in town_info.apply(lambda x : x['si_code']*1000000+x['gu_code']*1000+x['dong_code'], axis=1):
    print("%d code insert start" % (code))
    apt = getAptInfo(code)
    print(apt)
    if apt.empty :
        continue
    apt.to_sql('apt_code_tbl2', kb_db, index=False, if_exists='append' )
    print("%d code insert rows are %d" % (code,apt.count()[0]))

11680103 code insert start
      BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE   KAPT_NAME  \
0   1168010300  서울특별시 강남구 개포동               0  A13593901   LG개포자이아파트   
1   1168010300  서울특별시 강남구 개포동               0  A13593908   SH공사대치1단지   
2   1168010300  서울특별시 강남구 개포동               0  A13524006  개포2차 현대아파트   
3   1168010300  서울특별시 강남구 개포동               0  A13580602     개포경남아파트   
4   1168010300  서울특별시 강남구 개포동               0  A13524009     개포대치2단지   
5   1168010300  서울특별시 강남구 개포동               0  A13580801        개포시영   
6   1168010300  서울특별시 강남구 개포동               0  A13524004      개포우성3차   
7   1168010300  서울특별시 강남구 개포동               0  A13580002      개포우성8차   
8   1168010300  서울특별시 강남구 개포동               0  A13580601      개포우성9차   
9   1168010300  서울특별시 강남구 개포동               0  A13524011     개포주공1단지   
10  1168010300  서울특별시 강남구 개포동               0  A13580204     개포주공2단지   
11  1168010300  서울특별시 강남구 개포동               0  A13580103     개포주공3단지   
12  1168010300  서울특별시 강남구 개포동        

     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE       KAPT_NAME  \
0  1168011500  서울특별시 강남구 수서동               0  A10027446        강남더샵포레스트   
1  1168011500  서울특별시 강남구 수서동               0  A13552002         강남데시앙포레   
2  1168011500  서울특별시 강남구 수서동               0  A13552001  서울주택도시공사 수서6단지   
3  1168011500  서울특별시 강남구 수서동               0  A13522007          수서까치마을   
4  1168011500  서울특별시 강남구 수서동               0  A13588601            수서동익   
5  1168011500  서울특별시 강남구 수서동               0  A13522004            수서삼성   
6  1168011500  서울특별시 강남구 수서동               0  A13522003            수서삼익   
7  1168011500  서울특별시 강남구 수서동               0  A13522006           수서신동아   
8  1168011500  서울특별시 강남구 수서동               0  A13522008            수서주공   
9  1168011500  서울특별시 강남구 수서동               0  A13588402           수서한아름   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM            X            Y  
0       201607          201608      None  208354.3905  442097.3270  
1       201408          201409      

     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE    KAPT_NAME  \
0  1168011200  서울특별시 강남구 자곡동               0  A13519008      LH강남3단지   
1  1168011200  서울특별시 강남구 자곡동               0  A10028172  LH강남아이파크아파트   
2  1168011200  서울특별시 강남구 자곡동               0  A10027985    LH강남힐스테이트   
3  1168011200  서울특별시 강남구 자곡동               0  A13520002      강남한양수자인   
4  1168011200  서울특별시 강남구 자곡동               0  A13520003      래미안강남힐즈   
5  1168011200  서울특별시 강남구 자곡동               0  A13520001        래미안포레   
6  1168011200  서울특별시 강남구 자곡동               0  A13520004   엘에이치강남브리즈힐   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM              X              Y  
0       201311          201312      None  207983.368490  440956.093805  
1       201410          201411      None  208913.661335  441390.292135  
2       201506          201507      None  207754.579798  441029.258984  
3       201403          201404      None            NaN            NaN  
4       201406          201407      None  208255.232834  4

     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE     KAPT_NAME  \
0  1174010800  서울특별시 강동구 성내동               0  A13484004  SK허브진주상복합아파트   
1  1174010800  서울특별시 강동구 성내동               0  A13484501      둔촌역청구아파트   
2  1174010800  서울특별시 강동구 성내동               0  A13484201    성내1차e-편한세상   
3  1174010800  서울특별시 강동구 성내동               0  A13403001    성내2차e-편한세상   
4  1174010800  서울특별시 강동구 성내동               0  A13403101          성내삼성   
5  1174010800  서울특별시 강동구 성내동               0  A13403002        성내성안청구   
6  1174010800  서울특별시 강동구 성내동               0  A13484102         성내코오롱   
7  1174010800  서울특별시 강동구 성내동               0  A13484003          성내현대   
8  1174010800  서울특별시 강동구 성내동               0  A10027354    올림픽파크한양수자인   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM            X            Y  
0       200410          200411      None  211666.3200  448111.8660  
1       199903          199904      None  211699.7050  447469.5300  
2       200311          200312      None  211501.5350  447895.

11305102 code insert rows are 17
11305103 code insert start
     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE     KAPT_NAME  \
0  1130510300  서울특별시 강북구 수유동               0  A14207202         래미안수유   
1  1130510300  서울특별시 강북구 수유동               0  A14207201  래미안수유제2관리사무소   
2  1130510300  서울특별시 강북구 수유동               0  A14278101       수유극동아파트   
3  1130510300  서울특별시 강북구 수유동               4  A14207203          수유벽산   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM            X            Y  
0       200111          200112      None  201534.3855  460262.1335  
1       200111          200112      None  201534.3855  460262.1335  
2       199203          199204      None  201002.1420  459941.4035  
3       199311          199312      None  201601.1800  460024.5200  
11305103 code insert rows are 4
11305104 code insert start
     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE KAPT_NAME  \
0  1130510400  서울특별시 강북구 우이동               0  A14209001      우이대우   
1  1130510400  서울특별시 강북구 우이동    

...

      BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE           KAPT_NAME  \
0   1138011400  서울특별시 은평구 진관동               0  A41279927     은평뉴타운구파발10단지1관리   
1   1138011400  서울특별시 은평구 진관동               0  A41279928     은평뉴타운구파발10단지2관리   
2   1138011400  서울특별시 은평구 진관동               7  A41279931     은평뉴타운구파발10단지3관리   
3   1138011400  서울특별시 은평구 진관동               0  A41279920       은평뉴타운구파발9-2단지   
4   1138011400  서울특별시 은평구 진관동               0  A41279921      은평뉴타운구파발9단지1관리   
5   1138011400  서울특별시 은평구 진관동               0  A41279932        은평뉴타운기자촌11단지   
6   1138011400  서울특별시 은평구 진관동               0  A41279913        은평뉴타운마고정11단지   
7   1138011400  서울특별시 은평구 진관동               0  A41279914         은평뉴타운마고정2단지   
8   1138011400  서울특별시 은평구 진관동               4  A41279912         은평뉴타운마고정3단지   
9   1138011400  서울특별시 은평구 진관동               0  A41279909    은평뉴타운박석고개13단지아파트   
10  1138011400  서울특별시 은평구 진관동               0  A41279910        은평뉴타운박석고개1단지   
11  1138011400  서울특별시 은평구 진관동           

     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE     KAPT_NAME  \
0  1111018700  서울특별시 종로구 무악동               0  A11081503          무악현대   
1  1111018700  서울특별시 종로구 무악동               0  A11008001        무악현대임대   
2  1111018700  서울특별시 종로구 무악동               0  A10027708  인왕산2차아이파크아파트   
3  1111018700  서울특별시 종로구 무악동               0  A11081302       인왕산아이파크   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM           X          Y  
0       200010          200011      None  196604.720  452785.07  
1       200010          200011      None  196361.059  452642.57  
2       201508          201509      None  196341.110  452648.85  
3       200807          200808      None  196394.830  452859.55  
11110187 code insert rows are 4
11110152 code insert start
Empty DataFrame
Columns: []
Index: []
11110184 code insert start
Empty DataFrame
Columns: []
Index: []
11110144 code insert start
Empty DataFrame
Columns: []
Index: []
11110115 code insert start
     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT

     BJD_CODE        BJD_NAME  ENERGY_B_COUNT  KAPT_CODE KAPT_NAME  \
0  1114017300  서울특별시 중구 만리동1가               0  A10080701  LIG서울역리가   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM            X           Y  
0       201201          201202      None  197011.6145  450397.732  
11140173 code insert rows are 1
11140174 code insert start
     BJD_CODE        BJD_NAME  ENERGY_B_COUNT  KAPT_CODE       KAPT_NAME  \
0  1114017400  서울특별시 중구 만리동2가               0  A10026881     서울역센트럴자이아파트   
1  1114017400  서울특별시 중구 만리동2가               0  A10026517  서울역한라비발디센트럴아파트   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM              X              Y  
0       201708          201709      None  196725.552959  450035.031803  
1       201801          201802      None            NaN            NaN  
11140174 code insert rows are 2
11140126 code insert start
Empty DataFrame
Columns: []
Index: []
11140127 code insert start
Empty DataFrame
Columns: []
Index: []
11140101 code insert start
Empty DataFrame
Columns: []

11140163 code insert rows are 2
11260105 code insert start
     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE   KAPT_NAME  \
0  1126010500  서울특별시 중랑구 망우동               1  A13123102     망우금호어울림   
1  1126010500  서울특별시 중랑구 망우동               0  A13123101        망우신원   
2  1126010500  서울특별시 중랑구 망우동               0  A13123001   예성그린캐슬아파트   
3  1126010500  서울특별시 중랑구 망우동               0  A13123002  중랑숲Liga아파트   
4  1126010500  서울특별시 중랑구 망우동               0  A13176102       중랑해모로   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM            X            Y  
0       200604          200605      None  209567.3380  455555.6500  
1       199903          199904      None  209508.6700  455300.7050  
2       200405          200406      None  208943.3900  454970.7770  
3       201208          201209      None  209201.3835  455558.7555  
4       200705          200706      None  209654.0950  455614.7300  
11260105 code insert rows are 5
11260101 code insert start
      BJD_CODE       BJD_NAME  ENERGY_B_C

11260106 code insert rows are 25
11260103 code insert start
     BJD_CODE       BJD_NAME  ENERGY_B_COUNT  KAPT_CODE KAPT_NAME  \
0  1126010300  서울특별시 중랑구 중화동               0  A13187501      중화극동   
1  1126010300  서울특별시 중랑구 중화동               0  A13187702   중화한신아파트   

  KAPT_USEDATE OCCU_FIRST_DATE OPEN_TERM           X            Y  
0       199104          199105      None  207328.264  455699.3135  
1       199710          199711      None  207156.330  454997.5050  
11260103 code insert rows are 2


In [31]:
apt_info=pd.read_sql("select * from apt_code_tbl2;",kb_db)
apt_info

,BJD_CODE,BJD_NAME,ENERGY_B_COUNT,KAPT_CODE,KAPT_NAME,KAPT_USEDATE,OCCU_FIRST_DATE,OPEN_TERM,X,Y
0,1168010300,서울특별시 강남구 개포동,0,A13593901,LG개포자이아파트,200406,200407,None,206706.200000,443790.735000
1,1168010300,서울특별시 강남구 개포동,0,A13593908,SH공사대치1단지,199111,199112,None,206587.710000,443561.235000
2,1168010300,서울특별시 강남구 개포동,0,A13524006,개포2차 현대아파트,198601,198602,None,204465.635000,442258.960000
3,1168010300,서울특별시 강남구 개포동,0,A13580602,개포경남아파트,198403,198404,None,204731.265000,442478.655000
4,1168010300,서울특별시 강남구 개포동,0,A13524009,개포대치2단지,199210,199211,None,206587.710000,443561.235000
5,1168010300,서울특별시 강남구 개포동,0,A13580801,개포시영,198403,198404,201604,204713.550000,441811.050000
6,1168010300,서울특별시 강남구 개포동,0,A13524004,개포우성3차,198412,198501,None,204881.660000,442417.840000
7,1168010300,서울특별시 강남구 개포동,0,A13580002,개포우성8차,198709,198710,None,205448.260000,442882.295000
8,1168010300,서울특별시 강남구 개포동,0,A13580601,개포우성9차,199101,199102,None,204987.545000,442514.155000
9,1168010300,서울특별시 강남구 개포동,0,A13524011,개포주공1단지,198206,198207,None,205105.090000,442077.710000


In [35]:
len(apt_info)

2392

In [37]:
apt_info[78:]['KAPT_CODE']

78      A13509101
79      A13509003
80      A13509004
81      A13509001
82      A13586602
83      A13509012
84      A13587603
85      A13509009
86      A13519005
87      A13519007
88      A10027558
89      A13519001
90      A13519004
91      A13519006
92      A13519002
93      A10027446
94      A13552002
95      A13552001
96      A13522007
97      A13588601
98      A13522004
99      A13522003
100     A13522006
101     A13522008
102     A13588402
103     A13589802
104     A13511101
105     A13511001
106     A13511003
107     A13511004
          ...    
2362    A13185607
2363    A13186305
2364    A13170401
2365    A13187306
2366    A13176901
2367    A13187201
2368    A13187305
2369    A13176902
2370    A13113006
2371    A13186801
2372    A13187302
2373    A13178101
2374    A13186708
2375    A13113004
2376    A13113003
2377    A13113001
2378    A13186905
2379    A13113002
2380    A13187406
2381    A13186503
2382    A13187004
2383    A13186504
2384    A13186502
2385    A13113008
2386    A1

In [38]:
# apt_details = pd.DataFrame(columns=['KAPT_CODE', 'KAPT_DONG_CNT','KAPT_USEDATE', 'KAPTD_WTIMESUB', 'SUBWAY_STATION', 'SUBWAY_LINE', 'CODE_HALL','KAPT_BCOMPANY', 'KAPTDA_CNT1', 'KAPTDA_CNT2', 'KAPTDA_CNT3', 'KAPTDA_CNT4'])

for code in apt_info[78:]['KAPT_CODE']:
    a = getAptDetailInfo(code)
    if not bool(a):
        continue
    info = extractAptInfo(a)
    if not bool(info):
        continue
    n=len(apt_details)
    apt_details.loc[n] = info
    print ("%s apt data collection completed(%d)" % (code,n))

A13509101 apt data collection completed(79)
A13509003 apt data collection completed(80)
A13509004 apt data collection completed(81)
A13509001 apt data collection completed(82)
A13586602 apt data collection completed(83)
A13509012 apt data collection completed(84)
A13587603 apt data collection completed(85)
A13509009 apt data collection completed(86)
A13519005 apt data collection completed(87)
A13519007 apt data collection completed(88)
A10027558 apt data collection completed(89)
A13519001 apt data collection completed(90)
A13519004 apt data collection completed(91)
A13519006 apt data collection completed(92)
A13519002 apt data collection completed(93)
A10027446 apt data collection completed(94)
A13552002 apt data collection completed(95)
A13552001 apt data collection completed(96)
A13522007 apt data collection completed(97)
A13588601 apt data collection completed(98)
A13522004 apt data collection completed(99)
A13522003 apt data collection completed(100)
A13522006 apt data collection c

A13402002 apt data collection completed(262)
A13402304 apt data collection completed(263)
A13486601 apt data collection completed(264)
A13486401 apt data collection completed(265)
A13487102 apt data collection completed(266)
A14272304 apt data collection completed(267)
A14272311 apt data collection completed(268)
A10026947 apt data collection completed(269)
A14277601 apt data collection completed(270)
A14272309 apt data collection completed(271)
A14272308 apt data collection completed(272)
A14282301 apt data collection completed(273)
A14272306 apt data collection completed(274)
A14272314 apt data collection completed(275)
A14280502 apt data collection completed(276)
A14272307 apt data collection completed(277)
A14272305 apt data collection completed(278)
A14272310 apt data collection completed(279)
A14210002 apt data collection completed(280)
A14210001 apt data collection completed(281)
A14272313 apt data collection completed(282)
A14272303 apt data collection completed(283)
A14281701 

A15786424 apt data collection completed(445)
A15786426 apt data collection completed(446)
A15701007 apt data collection completed(447)
A15701601 apt data collection completed(448)
A15701602 apt data collection completed(449)
A15701003 apt data collection completed(450)
A15701002 apt data collection completed(451)
A15701004 apt data collection completed(452)
A15770101 apt data collection completed(453)
A15788302 apt data collection completed(454)
A15788901 apt data collection completed(455)
A15791001 apt data collection completed(456)
A15788203 apt data collection completed(457)
A15770801 apt data collection completed(458)
A15792602 apt data collection completed(459)
A15108001 apt data collection completed(460)
A15108003 apt data collection completed(461)
A15192201 apt data collection completed(462)
A15180705 apt data collection completed(463)
A15105503 apt data collection completed(464)
A15178201 apt data collection completed(465)
A15178204 apt data collection completed(466)
A15105008 

A15285503 apt data collection completed(628)
A15284906 apt data collection completed(629)
A15284305 apt data collection completed(630)
A15284204 apt data collection completed(631)
A15205301 apt data collection completed(632)
A15286801 apt data collection completed(633)
A15283809 apt data collection completed(634)
A15205103 apt data collection completed(635)
A15286602 apt data collection completed(636)
A15288002 apt data collection completed(637)
A15286809 apt data collection completed(638)
A15285804 apt data collection completed(639)
A15286805 apt data collection completed(640)
A15205303 apt data collection completed(641)
A15205104 apt data collection completed(642)
A15288004 apt data collection completed(643)
A15286802 apt data collection completed(644)
A15286807 apt data collection completed(645)
A15205108 apt data collection completed(646)
A15205508 apt data collection completed(647)
A15283902 apt data collection completed(648)
A15283801 apt data collection completed(649)
A15205513 

A13983105 apt data collection completed(811)
A13983004 apt data collection completed(812)
A13971502 apt data collection completed(813)
A13920706 apt data collection completed(814)
A13920703 apt data collection completed(815)
A13920707 apt data collection completed(816)
A13982704 apt data collection completed(817)
A13921003 apt data collection completed(818)
A13921005 apt data collection completed(819)
A13920209 apt data collection completed(820)
A13981402 apt data collection completed(821)
A13920202 apt data collection completed(822)
A13983608 apt data collection completed(823)
A13981304 apt data collection completed(824)
A13982005 apt data collection completed(825)
A13982002 apt data collection completed(826)
A13994302 apt data collection completed(827)
A13920205 apt data collection completed(828)
A13983707 apt data collection completed(829)
A13983709 apt data collection completed(830)
A13983712 apt data collection completed(831)
A13974101 apt data collection completed(832)
A13974102 

...

A13309101 apt data collection completed(1531)
A13380301 apt data collection completed(1532)
A13380302 apt data collection completed(1533)
A13380303 apt data collection completed(1534)
A10027602 apt data collection completed(1535)
A13380602 apt data collection completed(1536)
A13380703 apt data collection completed(1537)
A13379001 apt data collection completed(1538)
A13309301 apt data collection completed(1539)
A13309404 apt data collection completed(1540)
A13378101 apt data collection completed(1541)
A13309402 apt data collection completed(1542)
A13380901 apt data collection completed(1543)
A13378102 apt data collection completed(1544)
A13380803 apt data collection completed(1545)
A10026470 apt data collection completed(1546)
A13370801 apt data collection completed(1547)
A13305005 apt data collection completed(1548)
A13305002 apt data collection completed(1549)
A13381604 apt data collection completed(1550)
A13305006 apt data collection completed(1551)
A13305007 apt data collection comp

A13676704 apt data collection completed(1710)
A13610103 apt data collection completed(1711)
A13610003 apt data collection completed(1712)
A13610005 apt data collection completed(1713)
A13671209 apt data collection completed(1714)
A13671208 apt data collection completed(1715)
A13671212 apt data collection completed(1716)
A13671211 apt data collection completed(1717)
A13671213 apt data collection completed(1718)
A13671204 apt data collection completed(1719)
A13671205 apt data collection completed(1720)
A13671210 apt data collection completed(1721)
A13671207 apt data collection completed(1722)
A13686302 apt data collection completed(1723)
A13671203 apt data collection completed(1724)
A13671206 apt data collection completed(1725)
A13609001 apt data collection completed(1726)
A13613012 apt data collection completed(1727)
A13613009 apt data collection completed(1728)
A13613011 apt data collection completed(1729)
A13613007 apt data collection completed(1730)
A13613004 apt data collection comp

A15805105 apt data collection completed(1889)
A10027849 apt data collection completed(1890)
A15880204 apt data collection completed(1891)
A15882104 apt data collection completed(1892)
A15882103 apt data collection completed(1893)
A15870101 apt data collection completed(1894)
A15805502 apt data collection completed(1895)
A15805002 apt data collection completed(1896)
A15882008 apt data collection completed(1897)
A15882006 apt data collection completed(1898)
A15870102 apt data collection completed(1899)
A15805102 apt data collection completed(1900)
A15805114 apt data collection completed(1901)
A15805111 apt data collection completed(1902)
A15878601 apt data collection completed(1903)
A15809603 apt data collection completed(1904)
A15809502 apt data collection completed(1905)
A15809201 apt data collection completed(1906)
A15809401 apt data collection completed(1907)
A15883803 apt data collection completed(1908)
A15809001 apt data collection completed(1909)
A15809403 apt data collection comp

A15010303 apt data collection completed(2068)
A15010306 apt data collection completed(2069)
A15010302 apt data collection completed(2070)
A15010307 apt data collection completed(2071)
A15010304 apt data collection completed(2072)
A15010301 apt data collection completed(2073)
A15010305 apt data collection completed(2074)
A15086702 apt data collection completed(2075)
A15086601 apt data collection completed(2076)
A15086603 apt data collection completed(2077)
A15010501 apt data collection completed(2078)
A15010502 apt data collection completed(2079)
A15010602 apt data collection completed(2080)
A15010601 apt data collection completed(2081)
A15088915 apt data collection completed(2082)
A15088614 apt data collection completed(2083)
A15089307 apt data collection completed(2084)
A15001012 apt data collection completed(2085)
A15001019 apt data collection completed(2086)
A15001005 apt data collection completed(2087)
A15001016 apt data collection completed(2088)
A15076301 apt data collection comp

A41279903 apt data collection completed(2247)
A41279908 apt data collection completed(2248)
A10026414 apt data collection completed(2249)
A41279916 apt data collection completed(2250)
A41279917 apt data collection completed(2251)
A41279918 apt data collection completed(2252)
A41279915 apt data collection completed(2253)
A41279923 apt data collection completed(2254)
A41279922 apt data collection completed(2255)
A41279925 apt data collection completed(2256)
A41279929 apt data collection completed(2257)
A41279930 apt data collection completed(2258)
A41279924 apt data collection completed(2259)
A10026842 apt data collection completed(2260)
A10027118 apt data collection completed(2261)
A10027105 apt data collection completed(2262)
A10026940 apt data collection completed(2263)
A11007001 apt data collection completed(2264)
A11052201 apt data collection completed(2265)
A11081503 apt data collection completed(2266)
A11008001 apt data collection completed(2267)
A10027708 apt data collection comp

In [44]:
apt_details = apt_details.drop_duplicates()

In [45]:
apt_details

,KAPT_CODE,KAPT_DONG_CNT,KAPT_USEDATE,KAPTD_WTIMESUB,SUBWAY_STATION,SUBWAY_LINE,CODE_HALL,KAPT_BCOMPANY,KAPTDA_CNT1,KAPTDA_CNT2,KAPTDA_CNT3,KAPTDA_CNT4
0,A13593901,4,2004-06-17,5~10분이내,대청,3호선,계단식,화엄건설,0,0,84,128
1,A13593908,8,1991-11-20,5분이내,대모산,중앙선,복도식,SH공사,1623,0,0,0
2,A13524006,13,1986-01-31,10~15분이내,매봉,3호선,계단식,현대산업개발(주),0,112,230,216
3,A13580602,9,1984-03-08,5~10분이내,구룡,"3호선, 분당선",계단식,경남건설,0,3,387,288
4,A13524009,11,1992-10-14,-,-,3호선,복도식,성원건설.진덕산업주식회사,1753,0,0,0
5,A13580801,30,1984-03-20,-,-,-,계단식,"대우,한라",1970,0,0,0
6,A13524004,5,1984-12-31,5~10분이내,도곡 구룡,"3호선, 분당선",혼합식,우성건설,0,0,285,120
7,A13580002,3,1987-09-21,5분이내,"개포동역,구룡역",분당선,혼합식,우성건설,0,261,0,0
8,A13580601,2,1991-01-25,5~10분이내,도곡,"3호선, 분당선",계단식,우성건설,0,232,0,0
9,A13524011,124,1982-06-04,10~15분이내,"매봉, 구룡","3호선, 분당선",계단식,대한주택공사,4975,65,0,0


In [46]:
apt_details.to_sql('apt_info_tbl', kb_db, index=False, if_exists='append' )

In [41]:
# apt 거래 내역 정보 
def readAptData( dongcode, danjicode, year, quarter ) :
    url = "http://rt.molit.go.kr/rtApt.do?cmd=getTradeAptLocal&dongCode=%d&danjiCode=ALL&srhYear=%d&srhPeriod=%d&gubunRadio2=1" % ( dongcode, year, quarter )
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    apt_meta = pd.DataFrame(data['danjiList'])
    apt_data = pd.DataFrame(data['detailList'])
    return apt_meta, apt_data

In [49]:
with open('./MERGE/data_without_error_1203_full_name.txt', 'rb') as f:
    full = pickle.load(f)

In [87]:
full.columns.values

array(['deal_price', 'deal_year', 'deal_month', 'full_loc_code',
       'loc_code_x', 'dong', 'ji_bun', 'apt_name_y', 'area',
       'current_floor', 'total_unit', 'total_building', 'max_floor',
       'min_floor', 'total_parking', 'unit_parking', 'completed_date',
       'conv_lists', 'medi_lists', 'park_lists', 'road_lists',
       'kindergarten', 'elementary_school', 'middle_school',
       'high_school', 'university', 'etc_lists', 'S-Line1', 'S-Line2',
       'S-Line3', 'S-Line4', 'S-Line5', 'S-Line6', 'S-Line7', 'S-Line8',
       'S-Line9', 'S-LineBD', 'S-LineNBD', 'S-LineGC', 'S-LineJA',
       'S-LineAir', 'distance', 'full_address', 'price_yard', 'deal_date',
       'full_addr', 'entirelawcode', 'yearmonth', '법정동코드', '법정동명', '폐지여부',
       'transaction_year_month', 'transaction_date', 'year_of_completion',
       'exclusive_use_area', 'floor', 'latitude', 'longitude',
       'address_by_law', 'total_parking_capacity_in_site',
       'total_household_count_in_sites',
       'apa

In [91]:
full[(full['full_address'] == '서울특별시 강남구 개포동 12') & (full['apt_name_y'] == '대청')]

,deal_price,deal_year,deal_month,full_loc_code,loc_code_x,dong,ji_bun,apt_name_y,area,current_floor,total_unit,total_building,max_floor,min_floor,total_parking,unit_parking,completed_date,conv_lists,medi_lists,park_lists,road_lists,kindergarten,elementary_school,middle_school,high_school,university,etc_lists,S-Line1,S-Line2,S-Line3,S-Line4,S-Line5,S-Line6,S-Line7,S-Line8,S-Line9,S-LineBD,S-LineNBD,S-LineGC,S-LineJA,S-LineAir,distance,full_address,price_yard,deal_date,full_addr,entirelawcode,yearmonth,법정동코드,법정동명,폐지여부,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,heat_type,heat_fuel,room_count,bathroom_count,front_door_structure,transaction_real_price,cont_rank,com_year,com_mon,avg_price,sub_num,bldg_age,lv_fg
746575,159000000,2006,1,1168001,11680,1,12,대청,33,1,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,479.204340,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,1~10,1992,33.18,1,37.494686,127.075402,1168010300,1753.0,1753,11,15.0,15.0,district,cogeneration,2.0,1.0,corridor,159000000,350,1992,10,4792043,2,14,0.066667
746577,248000000,2006,1,1168001,11680,1,12,대청,39,5,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,627.371616,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,1~10,1992,39.53,5,37.494987,127.077138,1168010300,700.0,822,6,15.0,13.0,district,cogeneration,3.0,1.0,corridor,248000000,350,1992,10,6273716,2,14,0.333333
746579,333000000,2006,1,1168001,11680,1,12,대청,49,11,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,667.870036,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,1~10,1992,49.86,11,37.494686,127.075402,1168010300,1753.0,1753,11,15.0,15.0,district,cogeneration,2.0,1.0,corridor,333000000,350,1992,10,6678700,2,14,0.733333
746581,247000000,2006,1,1168001,11680,1,12,대청,39,12,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,624.841892,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,1~10,1992,39.53,12,37.494686,127.075402,1168010300,1753.0,1753,11,15.0,15.0,district,cogeneration,2.0,1.0,corridor,247000000,350,1992,10,6248418,2,14,0.800000
746583,247000000,2006,1,1168001,11680,1,12,대청,39,4,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,624.841892,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,1~10,1992,39.53,4,37.494686,127.075402,1168010300,1753.0,1753,11,15.0,15.0,district,cogeneration,2.0,1.0,corridor,247000000,350,1992,10,6248418,2,14,0.266667
746585,337000000,2006,1,1168001,11680,1,12,대청,51,8,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,659.233177,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,11~20,1992,51.12,8,37.494987,127.077138,1168010300,700.0,822,6,15.0,13.0,district,cogeneration,2.0,1.0,corridor,337000000,350,1992,10,6592331,2,14,0.533333
746587,252000000,2006,1,1168001,11680,1,12,대청,39,8,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,637.490514,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,11~20,1992,39.53,8,37.494686,127.075402,1168010300,1753.0,1753,11,15.0,15.0,district,cogeneration,2.0,1.0,corridor,252000000,350,1992,10,6374905,2,14,0.533333
746589,330000000,2006,1,1168001,11680,1,12,대청,49,6,822,6,15,13,700,0,1992.10,2,0,4,1,2,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,580,서울특별시 강남구 개포동 12,661.853189,2006-01-01,서울특별시 강남구 개포동 12,서울특별시 강남구 개포동,200601,1168010300,서울특별시 강남구 개포동,존재,200601,11~20,1992,49.86,6,37.494686,

In [129]:
with open('./Dataset/Price/deal_whole_data.txt', 'rb') as f:
    price = pickle.load(f)

In [130]:
price

[{'거래금액': '29,500',
  '건축년도': '1983',
  '년': '2006',
  '법정동': '당주동',
  '아파트': '세종',
  '월': '1',
  '일': '11~20',
  '전용면적': '93.4',
  '지번': '100',
  '지역코드': '11110',
  '층': '6'},
 {'거래금액': '15,741',
  '건축년도': '1998',
  '년': '2006',
  '법정동': '익선동',
  '아파트': '운현신화타워',
  '월': '1',
  '일': '1~10',
  '전용면적': '57.18',
  '지번': '30-6',
  '지역코드': '11110',
  '층': '12'},
 {'거래금액': '23,198',
  '건축년도': '1998',
  '년': '2006',
  '법정동': '익선동',
  '아파트': '운현신화타워',
  '월': '1',
  '일': '1~10',
  '전용면적': '84.26',
  '지번': '30-6',
  '지역코드': '11110',
  '층': '6'},
 {'거래금액': '23,198',
  '건축년도': '1998',
  '년': '2006',
  '법정동': '익선동',
  '아파트': '운현신화타워',
  '월': '1',
  '일': '11~20',
  '전용면적': '84.26',
  '지번': '30-6',
  '지역코드': '11110',
  '층': '14'},
 {'거래금액': '21,535',
  '건축년도': '1998',
  '년': '2006',
  '법정동': '익선동',
  '아파트': '운현신화타워',
  '월': '1',
  '일': '11~20',
  '전용면적': '78.23',
  '지번': '30-6',
  '지역코드': '11110',
  '층': '15'},
 {'거래금액': '21,500',
  '건축년도': '1999',
  '년': '2006',
  '법정동': '명륜2가',
  '아파트': '아남3',
  '월

In [137]:
price = pd.DataFrame(price)

In [201]:
with open('/Users/tax_k/Projects/AI-DL/KB/Dataset/Information/deal_whole_data2.txt', 'rb') as f:
    addition = pickle.load(f)

In [202]:
addition[909: 942]

['거래금액',
 '건축년도',
 '년',
 '법정동',
 '아파트',
 '월',
 '일',
 '전용면적',
 '지번',
 '지역코드',
 '층',
 '거래금액',
 '건축년도',
 '년',
 '법정동',
 '아파트',
 '월',
 '일',
 '전용면적',
 '지번',
 '지역코드',
 '층',
 '거래금액',
 '건축년도',
 '년',
 '법정동',
 '아파트',
 '월',
 '일',
 '전용면적',
 '지번',
 '지역코드',
 '층']

In [203]:
del addition[909: 942]

In [151]:
addi = pd.DataFrame(addition[0], index=[0])

In [152]:
addi

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"140,000",2008,2018,사직동,광화문풍림스페이스본(9-1),11,21~30,153.42,9-1,11110,7


In [155]:
cols = addi.columns.values

In [156]:
df_ = pd.DataFrame(columns=cols)

In [210]:
df_.iloc[2782]

거래금액    78,000 
건축년도    2009   
년       2018   
법정동     북가좌동   
아파트     DMC아이파크
월       12     
일       1~10   
전용면적    84.92  
지번      458    
지역코드    11410  
층       11     
Name: 936, dtype: object

In [219]:
del addition[937: 948]

In [230]:
del addition[946: 968]

In [231]:
for i in range(0, len(addition)):
    tmp = pd.DataFrame(addition[i], index=[i])
    df_ = pd.concat([df_, tmp])

In [233]:
df_ = df_.drop_duplicates()

In [234]:
df_

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"140,000",2008,2018,사직동,광화문풍림스페이스본(9-1),11,21~30,153.42,9-1,11110,7
1,"68,000",2006,2018,인의동,효성쥬얼리시티,11,1~10,84.64,48-2,11110,13
2,"35,000",2017,2018,효제동,포레스트힐시티,11,21~30,35.8451,65-2,11110,13
3,"47,800",1999,2018,명륜2가,아남3,11,1~10,44.37,237,11110,8
4,"41,000",1993,2018,창신동,창신쌍용2,11,11~20,54.7,703,11110,11
5,"8,450",1971,2018,행촌동,대성맨션,11,1~10,19.27,41-1,11110,1
6,"79,000",2003,2018,평창동,크래스빌,11,11~20,187.9,46-1,11110,3
7,"104,519",2009,2018,평창동,엘리시아,11,11~20,213.07,66-1,11110,4
9,"170,000",2010,2018,남대문로5가,남산트라팰리스,11,11~20,142.23,827,11140,31
10,"72,500",1999,2018,신당동,삼성,11,1~10,84.96,843,11140,2


In [149]:
bbb = addition

In [199]:
del bbb[909: 942]

In [195]:
lt = [1, 2, 3, 4]

In [196]:
lt[1:2]

[2]

In [197]:
del lt[1:2]

In [198]:
lt

[1, 3, 4]

In [173]:
bbb = pd.DataFrame(bbb, index=[0])

AttributeError: 'str' object has no attribute 'keys'

In [159]:
df_

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
0,"140,000",2008,2018,사직동,광화문풍림스페이스본(9-1),11,21~30,153.42,9-1,11110,7
1,"68,000",2006,2018,인의동,효성쥬얼리시티,11,1~10,84.64,48-2,11110,13
2,"35,000",2017,2018,효제동,포레스트힐시티,11,21~30,35.8451,65-2,11110,13
3,"47,800",1999,2018,명륜2가,아남3,11,1~10,44.37,237,11110,8
4,"41,000",1993,2018,창신동,창신쌍용2,11,11~20,54.7,703,11110,11
5,"8,450",1971,2018,행촌동,대성맨션,11,1~10,19.27,41-1,11110,1
6,"79,000",2003,2018,평창동,크래스빌,11,11~20,187.9,46-1,11110,3
7,"104,519",2009,2018,평창동,엘리시아,11,11~20,213.07,66-1,11110,4
8,"104,519",2009,2018,평창동,엘리시아,11,11~20,213.07,66-1,11110,4
9,"170,000",2010,2018,남대문로5가,남산트라팰리스,11,11~20,142.23,827,11140,31


In [86]:
price[price['아파트'] == '삼익대청아파트']

,거래금액,건축년도,년,법정동,아파트,월,일,전용면적,지번,지역코드,층
4271,"24,800",1992,2006,개포동,삼익대청아파트,1,1~10,39.53,12,11680,5
4295,"33,700",1992,2006,개포동,삼익대청아파트,1,11~20,51.12,12,11680,8
4315,"44,000",1992,2006,개포동,삼익대청아파트,1,11~20,60,12,11680,14
4321,"23,720",1992,2006,개포동,삼익대청아파트,1,11~20,39.53,12,11680,15
4323,"44,000",1992,2006,개포동,삼익대청아파트,1,11~20,60,12,11680,7
4357,"45,250",1992,2006,개포동,삼익대청아파트,1,11~20,60,12,11680,9
4380,"22,300",1992,2006,개포동,삼익대청아파트,1,21~31,39.53,12,11680,15
12246,"25,700",1992,2006,개포동,삼익대청아파트,2,1~10,39.53,12,11680,10
12269,"44,000",1992,2006,개포동,삼익대청아파트,2,11~20,60,12,11680,5
12270,"41,000",1992,2006,개포동,삼익대청아파트,2,11~20,60,12,11680,1


In [235]:
price = pd.concat([price, df_])

In [236]:
price.columns = ['deal_price', 'build_y', 'deal_y', 'address_by_law', 'apt_name', 'deal_m', 'deal_d', 'supl_area', 'jibun','loc_code', 'floor']

In [237]:
price

,deal_price,build_y,deal_y,address_by_law,apt_name,deal_m,deal_d,supl_area,jibun,loc_code,floor
0,"29,500",1983,2006,당주동,세종,1,11~20,93.4,100,11110,6
1,"15,741",1998,2006,익선동,운현신화타워,1,1~10,57.18,30-6,11110,12
2,"23,198",1998,2006,익선동,운현신화타워,1,1~10,84.26,30-6,11110,6
3,"23,198",1998,2006,익선동,운현신화타워,1,11~20,84.26,30-6,11110,14
4,"21,535",1998,2006,익선동,운현신화타워,1,11~20,78.23,30-6,11110,15
5,"21,500",1999,2006,명륜2가,아남3,1,11~20,47.43,237,11110,6
6,"20,000",1999,2006,명륜2가,아남3,1,11~20,44.37,237,11110,8
7,"16,800",1992,2006,창신동,창신쌍용1,1,11~20,54.7,702,11110,8
8,"16,500",1993,2006,창신동,창신쌍용2,1,11~20,64.66,703,11110,11
9,"28,000",1993,2006,창신동,창신쌍용2,1,11~20,106.62,703,11110,7


In [107]:
bbb = price.head()

In [114]:
ccc = bbb.to_dict('index')

{'거래금액': '15,741',
 '건축년도': '1998',
 '년': '2006',
 '법정동': '익선동',
 '아파트': '운현신화타워',
 '월': '1',
 '일': '1~10',
 '전용면적': '57.18',
 '지번': '30-6',
 '지역코드': '11110',
 '층': '12'}

In [111]:
jsonstr = bbb.to_json(orient='index', force_ascii=False)

In [112]:
jsonstr

'{"0":{"거래금액":"29,500","건축년도":"1983","년":"2006","법정동":"당주동","아파트":"세종","월":"1","일":"11~20","전용면적":"93.4","지번":"100","지역코드":"11110","층":"6"},"1":{"거래금액":"15,741","건축년도":"1998","년":"2006","법정동":"익선동","아파트":"운현신화타워","월":"1","일":"1~10","전용면적":"57.18","지번":"30-6","지역코드":"11110","층":"12"},"2":{"거래금액":"23,198","건축년도":"1998","년":"2006","법정동":"익선동","아파트":"운현신화타워","월":"1","일":"1~10","전용면적":"84.26","지번":"30-6","지역코드":"11110","층":"6"},"3":{"거래금액":"23,198","건축년도":"1998","년":"2006","법정동":"익선동","아파트":"운현신화타워","월":"1","일":"11~20","전용면적":"84.26","지번":"30-6","지역코드":"11110","층":"14"},"4":{"거래금액":"21,535","건축년도":"1998","년":"2006","법정동":"익선동","아파트":"운현신화타워","월":"1","일":"11~20","전용면적":"78.23","지번":"30-6","지역코드":"11110","층":"15"}}'

In [104]:
with open('df.json', 'w', encoding='utf-8') as file:
    price.to_json(file, orient='index', force_ascii=False)

In [101]:
import json